## Introduction

In [ ]:
import RMT

import torch
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [ ]:
torchvision.datasets.CIFAR100(root="fig/datasets", download=True)

In [ ]:
dataset = torchvision.datasets.CIFAR10(root="fig/datasets", transform=torchvision.transforms.ToTensor())
dataset_data = np.array(dataset.data, dtype=np.float32)
dataset_targets = np.array(dataset.targets)
display(dataset.data.shape)
class_to_data = {name: dataset_data[dataset_targets == l] for name, l in dataset.class_to_idx.items()}
# show number of images (and tensor shape) per class
display({k: v.shape for k, v in class_to_data.items()})
layer_size = np.prod(dataset.data.shape[1:])
display(layer_size)
norm_data_dict = {
    k: (v / np.mean(v**2, tuple(range(1, len(v.shape))), keepdims=True)**0.5)
    for k, v in class_to_data.items()
}

## Demonstration on empirical MLPs

In [ ]:
outputs_dict = {}

alpha = 1.5
sigma_W = 1.5

fp_norm = (
    RMT.MFT_map(RMT.q_star_MC(alpha, sigma_W)[-1], alpha, sigma_W, usetqdm=False)[
        "postact_sq_mean"
    ][0]
    ** 0.5
)

for label, data in tqdm(norm_data_dict.items()):
    num_items = 2 # len(data)
    # flatten the images and scale by fixed point norm
    x0 = data[:num_items].reshape(num_items, -1) * fp_norm
    xs = RMT.MLP(
        torch.tensor(x0),
        10,
        alpha,
        sigma_W,
        seed=42,
        fast=True,
        usetqdm=False,
    )["postact"]
    norms = (np.array([x0, *xs]) ** 2).mean(-1) ** 0.5

    outputs_dict[label] = norms

In [ ]:
color_dict = {label: plt.cm.tab10(i) for i, label in enumerate(norm_data_dict.keys())}
for label, norms in outputs_dict.items():
    plt.plot(
        norms.mean(-1),
        "-o",
        # alpha=x0.shape[0] ** -0.5,
        color=color_dict[label],
        label=label,
    )
    plt.fill_between(
        np.arange(len(norms)),
        np.quantile(norms, 0.01, axis=-1),
        np.quantile(norms, 0.99, axis=-1),
        color=color_dict[label],
        alpha=0.5,
    )

plt.ylim([0.25, 0.75])
# plt.legend()
plt.show()

## MFT demonstration
takes longer than the MLP (about 7m33s on a Tesla M4 for CIFAR-10, 10 layers)

In [ ]:
outputs_dict = {}

alpha = 1.5
sigma_W = 1.5

fp_norm = (
    RMT.MFT_map(RMT.q_star_MC(alpha, sigma_W)[-1], alpha, sigma_W, usetqdm=False)[
        "postact_sq_mean"
    ][0]
    ** 0.5
)

# gpu memory bottleneck due to `RMT.MFT_map::postact_samples`
max_num_items_in_batch = 100

for label, data in tqdm(norm_data_dict.items()):
    num_items = 100  # len(data)
    x0 = data[:num_items].reshape(num_items, -1) * fp_norm
    q0 = sigma_W**alpha * (abs(x0) ** alpha).mean(-1)
    MFT_maps = []
    for q0_chunk in tqdm(
        np.array_split(q0, max(1, num_items // max_num_items_in_batch), axis=0),
        leave=False,
    ):
        torch.manual_seed(42)
        val = RMT.MFT_map(
            q0_chunk,
            alpha,
            sigma_W,
            num_layers=10,
            agg_postact=lambda x: {
                "sq_mean": (x**2).mean(-1),
                "alpha_mean": (abs(x) ** alpha).mean(-1),
            },
            usetqdm=False,
        )
        MFT_maps.append(val)
    # average over Monte-Carlo realisations (axis=2), then concatenate chunks of inputs (axis=1)
    agg_stats = {
        k: np.concatenate([np.mean(mft[k], axis=2) for mft in MFT_maps], axis=1)
        for k in MFT_maps[0]
    }
    norms = np.array([(x0**2).mean(-1), *agg_stats["postact_sq_mean"]]) ** 0.5
    alpha_norms = np.array(
        [(abs(x0) ** alpha).mean(-1), *agg_stats["postact_alpha_mean"]]
    ) ** (1 / alpha)
    qs = agg_stats["q_val"]

    outputs_dict[label] = norms

In [ ]:
# np.savez('fig/mixed_selectivity/outputs_MLP.npz', **outputs_dict)
outputs_dict = np.load('fig/mixed_selectivity/outputs_MLP.npz')

In [ ]:
fig, ax = plt.subplots()

color_dict = {label: plt.cm.tab10(i) for i, label in enumerate(norm_data_dict.keys())}
for label, norms in outputs_dict.items():
    ax.plot(
        norms.mean(-1),
        "-o",
        # alpha=x0.shape[0] ** -0.5,
        color=color_dict[label],
        label=label,
    )
    ax.fill_between(
        np.arange(len(norms)),
        np.quantile(norms, 0.01, axis=-1),
        np.quantile(norms, 0.99, axis=-1),
        color=color_dict[label],
        alpha=0.5,
    )

ax.set(
    xlabel='layer',
    ylabel='postact_norm',
    # ylim=(0.25, 0.75),
)
# ax.legend()
plt.show()

# Testing the agg stats function for the cluster

In [ ]:
from mixed_selectivity import MFT_map

import torch
import numpy as np
import matplotlib.pyplot as plt

torch.set_default_device("cpu")
result = MFT_map(
    dataset_name="CIFAR10",
    alpha=1.5,
    sigma_W=1.5,
    sigma_b=0.0,
    num_layers=10,
    chunk_size=100,
    seed=42,
    num_inputs=550,
)

print({k: v.shape for k, v in result.items()})

vals = result["postact_sq_mean"]
plt.plot(range(vals.shape[0]), vals.mean(axis=1), "-o")
plt.fill_between(
    range(vals.shape[0]),
    np.quantile(vals, 0.1, axis=1),
    np.quantile(vals, 0.9, axis=1),
    alpha=0.5,
)